In [42]:
class Operation():
    def __init__(self, input_nodes = []):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
            print(node)
        print(self)
            
        _default_graph.operations.append(self)
            
            
    def compute(self):
        pass
        

In [39]:
class add(Operation):
    def __init__(self, x, y):
        super().__init__([x ,y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        return x+y

In [5]:
class multiply(Operation):
    def __init__(self, x, y):
        super().__init__([x ,y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        
        return x*y

In [6]:
class matmul(Operation):
    def __init__(self, x, y):
        super().__init__([x ,y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        return x.dot(y)

In [8]:
class Placeholder():
    def __init__(self):
        self.output_nodes = []
        _default_graph.placeholders.append(self)

In [18]:
class Variable():
    def __init__(self, initial_value = None):
        self.value = initial_value
        self.output_nodes = []
        print('variable is', self.value, self.output_nodes, _default_graph)
        
        _default_graph.variables.append(self)

In [19]:
class Graph():
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        global _default_graph
        
        
        _default_graph = self
        print('graph now is everywhere', _default_graph)
        

In [12]:
g = Graph()

In [20]:
g.set_as_default()


In [17]:
_default_graph


In [21]:
A = Variable(10)

variable is 10 [] <__main__.Graph object at 0x111e31630>


In [22]:
b = Variable(1)

variable is 1 [] <__main__.Graph object at 0x111e31630>


In [23]:
x = Placeholder()

In [28]:
_default_graph.variables[0].value

10

In [32]:
_default_graph.placeholders[0].output_nodes

[]

In [40]:
y = multiply(A, x)

In [34]:
y

In [37]:
z = add(y, b)

In [41]:
_default_graph.operations

[]

In [44]:
y , z, _default_graph.operations

(<__main__.multiply at 0x112033278>, <__main__.add at 0x112027ef0>, [])

In [45]:
def traverse_postorder(operation):
    
    nodes_postorder = []
    
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
    recurse(operation)
    return nodes_postorder
            

In [54]:
class Session():
    def run(self, operation, feed_dict = {}):
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            
            if type(node)==Placeholder:
                node.output = feed_dict[node]
                
            elif type(node)==Variable:
                node.output = node.value
            
            else:
                node.inputs = [input_node.output for input_node in node.input_nodes]
                
                node.output = node.compute(*node.inputs)
                
            if type(node.output) == list:
                node.output = np.array(node.output)
                
        return operation.output
                
        
        
        
        
        
        
        
        
        

In [56]:
sess = Session()

In [58]:
result = sess.run(operation = z, feed_dict = {x:10})


AttributeError: 'multiply' object has no attribute 'output'